In [ ]:
from TensorflowModel import TensorflowModel
from Sent2VecModel import Sent2VecModel
import pickle
import faiss
from IPython.display import display, HTML
import re
import json
import sys
import numpy as np
import os
import base64

In [ ]:
# model path
model_path = '/home/seongjuk/project/meme-glossary/server/nlp/model/127_hgtki_lr9_ep11.bin'
# image root folder path - folder of folders of images
img_path = '/home/seongjuk/project/meme-glossary/tutorial/2_kumiko_cut_scenes'
# saved embedding path
saved_embedding_path = '/home/seongjuk/project/meme-glossary/tutorial/4_saved_embedding-hgtk.pickle'

In [ ]:
# Import Tensorflow model
model = TensorflowModel()
model.load_model(model_path)

In [ ]:
# Import sent2vec model
model = Sent2VecModel()
model.load_model(model_path)

In [ ]:
# load embedding
saved_embedding = pickle.load(open(saved_embedding_path, 'rb'))
index = faiss.IndexFlatIP(saved_embedding['embedding_dimension'])

print(np.shape(saved_embedding['embedding']))
print(saved_embedding['embedding_dimension'])
index.add(saved_embedding['embedding'])

In [ ]:
query_embedding = model.embed_sentence('감사')
print(np.shape(query_embedding))

In [ ]:
# korean decomposing model only
import hgtk
htext = hgtk.text.decompose('감사')
print(htext)
query_embedding = model.embed_sentence(htext)

In [ ]:
D, I = index.search(x=query_embedding, k=3)
sim_list = D[0].tolist()
result_index = I[0]
print(D)
print(I)

for sim, i in zip(sim_list, I[0]):
    # data = base64.b64encode(meme_file.read())
    # data = str(data)
    #display(HTML('''<img src="data:image/jpg;base64,''' + b64data + '''">'''))
    print(saved_embedding['text'][i])
    print(sim)

    jpg_path = os.path.join(os.path.abspath(img_path), saved_embedding['file_path'][i])

    with open(jpg_path, 'rb') as meme_file:
        # Encode image with base64 and str.
        data = base64.b64encode(meme_file.read())
        data = str(data)

        data = data.replace("'", '"')
        unpacked_b64data = re.search(r'\"(.*)\"', data).group(1)
        
        display(HTML('''<img src="data:image/jpg;base64,''' + unpacked_b64data + '''">'''))
